In [ ]:
!pip install git+https://github.com/tensorflow/docs
!pip install tensorflow
!pip install imutils
!pip install opencv-python
!pip install imageio

  Cloning https://github.com/tensorflow/docs to /tmp/pip-req-build-sylj9jj8
  Running command git clone --filter=blob:none --quiet https://github.com/tensorflow/docs /tmp/pip-req-build-sylj9jj8
  Resolved https://github.com/tensorflow/docs to commit 3688f3cff2685cfeab307c13435f77d2c96cf434
  Preparing metadata (setup.py) ... done


In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip /content/drive/MyDrive/VAD/Football_video_dataset.zip


Archive:  /content/drive/MyDrive/VAD/Football_video_dataset.zip
replace Football_video_dataset/test/Goal/goal (107).avi? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
from tensorflow_docs.vis import embed
from tensorflow import keras
from imutils import paths
from keras import layers
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import GRU, Bidirectional, Dropout, Dense
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os


In [ ]:
dataset_path = os.listdir('Football_video_dataset/train')

# Check if directories exist
train_dir = 'Football_video_dataset/train'
test_dir = 'Football_video_dataset/test'
if not os.path.exists(train_dir) or not os.path.exists(test_dir):
    raise FileNotFoundError("Training or testing directories do not exist.")

label_types = os.listdir('Football_video_dataset/train')
print (label_types)


['Goal', 'Loss', 'Happy']


In [ ]:
video_list = []

for item in dataset_path:

 # Get all the file names
 all_video_list = os.listdir('Football_video_dataset/train' + '/' +item)

 # Add them to the list
 for video in all_video_list:
    video_list.append((item, str('Football_video_dataset/train' + '/' +item) + '/' + video))

# Build a dataframe
train_df = pd.DataFrame(data=video_list, columns=['tag', 'video_name'])
print(train_df.head())
print(train_df.tail())
df = train_df.loc[:,['video_name','tag']]
df.to_csv('train.csv')


    tag                                       video_name
0  Goal  Football_video_dataset/train/Goal/goal (23).avi
1  Goal  Football_video_dataset/train/Goal/goal (66).avi
2  Goal  Football_video_dataset/train/Goal/goal (89).avi
3  Goal  Football_video_dataset/train/Goal/goal (18).avi
4  Goal   Football_video_dataset/train/Goal/goal (9).avi
       tag                                         video_name
313  Happy  Football_video_dataset/train/Happy/Happy (37).mp4
314  Happy  Football_video_dataset/train/Happy/Happy (70).mp4
315  Happy  Football_video_dataset/train/Happy/Happy (55).mp4
316  Happy  Football_video_dataset/train/Happy/Happy (32).mp4
317  Happy  Football_video_dataset/train/Happy/Happy (43).mp4


In [ ]:
dataset_path = os.listdir('Football_video_dataset/test')
print(dataset_path)

video_list = []

for item in dataset_path:
 # Get all the file names
 all_video_list = os.listdir('Football_video_dataset/test' + '/' +item)

 # Add them to the list
 for video in all_video_list:
    video_list.append((item, str('Football_video_dataset/test' + '/' +item) + '/' + video))

# Build a dataframe
test_df = pd.DataFrame(data=video_list, columns=['tag', 'video_name'])
print(test_df.head())
print(test_df.tail())

df = test_df.loc[:,['video_name','tag']]
df.to_csv('test.csv')


['Goal', 'Loss', 'Happy']
    tag                                       video_name
0  Goal  Football_video_dataset/test/Goal/goal (114).avi
1  Goal  Football_video_dataset/test/Goal/goal (119).avi
2  Goal  Football_video_dataset/test/Goal/goal (111).avi
3  Goal  Football_video_dataset/test/Goal/goal (117).avi
4  Goal  Football_video_dataset/test/Goal/goal (107).avi
      tag                                         video_name
73  Happy  Football_video_dataset/test/Happy/Happy (108).mp4
74  Happy  Football_video_dataset/test/Happy/Happy (122).mp4
75  Happy  Football_video_dataset/test/Happy/Happy (120).mp4
76  Happy  Football_video_dataset/test/Happy/Happy (116).mp4
77  Happy  Football_video_dataset/test/Happy/Happy (110).mp4


In [ ]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

print(f"Total videos for training: {len(train_df)}")
print(f"Total videos for testing: {len(test_df)}")



train_df.sample(10)

Total videos for training: 318
Total videos for testing: 78


,Unnamed: 0,video_name,tag
69,69,Football_video_dataset/train/Goal/goal (71).avi,Goal
112,112,Football_video_dataset/train/Loss/Loss (20).avi,Loss
226,226,Football_video_dataset/train/Happy/Happy (36).mp4,Happy
183,183,Football_video_dataset/train/Loss/Loss (72).avi,Loss
162,162,Football_video_dataset/train/Loss/Loss (79).avi,Loss
2,2,Football_video_dataset/train/Goal/goal (89).avi,Goal
298,298,Football_video_dataset/train/Happy/Happy (22).mp4,Happy
250,250,Football_video_dataset/train/Happy/Happy (86).mp4,Happy
43,43,Football_video_dataset/train/Goal/goal (8).avi,Goal
25,25,Football_video_dataset/train/Goal/goal (12).avi,Goal


In [ ]:
IMG_SIZE = 224

def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)


In [1]:
from keras import layers
from keras.applications import InceptionV3
import keras

IMG_SIZE = 224
NUM_CLASSES = 3

def build_feature_extractor():
    # Load InceptionV3 model with pre-trained weights
    base_model = InceptionV3(
        weights='imagenet',
        include_top=False,
        input_shape=(IMG_SIZE, IMG_SIZE, 3)
    )

    # Freeze the pre-trained layers
    for layer in base_model.layers:
        layer.trainable = False

    # Global average pooling layer to reduce spatial dimensions
    global_average_layer = layers.GlobalAveragePooling2D()(base_model.output)

    # Add custom dense layers for your task
    dense_layer = layers.Dense(256, activation='relu')(global_average_layer)
    dropout_layer = layers.Dropout(0.5)(dense_layer)

    # Output layer with the number of classes in your task
    output_layer = layers.Dense(NUM_CLASSES, activation='softmax')(dropout_layer)

    # Create the model
    model = keras.Model(inputs=base_model.input, outputs=output_layer, name='feature_extractor')

    return model

# Build the InceptionV3 feature extractor
feature_extractor = build_feature_extractor()


87910968/87910968 [==============================] - 1s 0us/step


In [ ]:
IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 30
MAX_SEQ_LENGTH = 20
NUM_FEATURES = 256
NUM_CLASSES = 3

In [ ]:
# Label processing
label_processor = Tokenizer()
labels = train_df["tag"].values
label_processor.fit_on_texts(labels)
train_labels_numeric = np.array(label_processor.texts_to_sequences(labels)) - 1

In [ ]:
def prepare_all_videos(df, folder, root_dir, MAX_SEQ_LENGTH, NUM_FEATURES):
    num_samples = len(df)
    video_paths = df["video_name"].values.tolist()

    # Take all class labels from train_df column named 'tag' and store in labels
    labels = df["tag"].values

    # Tokenize labels
    label_processor = Tokenizer()
    label_processor.fit_on_texts(labels)
    labels = np.array(label_processor.texts_to_sequences(labels)) - 1

    # Frame masks and frame features
    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool")
    frame_features = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

    # For each video
    for idx, path in enumerate(video_paths):
        frames = load_video(os.path.join(root_dir, path))
        frames = frames[None, ...]

        temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
        temp_frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

        # Extract features from frames
        for i in range(frames.shape[1]):
            temp_frame_features[0, i, :] = build_feature_extractor(frames[0, i, :])
            temp_frame_mask[0, i] = 1

        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()

    return [frame_features, frame_masks], labels


In [ ]:
root_dir = '/content/Football_video_dataset'

# Load and preprocess train data
train_data, _ = prepare_all_videos(train_df, "train", root_dir, MAX_SEQ_LENGTH, NUM_FEATURES)

# Load and preprocess test data
test_data, _ = prepare_all_videos(test_df, "test", root_dir, MAX_SEQ_LENGTH, NUM_FEATURES)

# Tokenize test labels
test_labels_numeric = np.array(label_processor.texts_to_sequences(test_df["tag"].values)) - 1

# Assuming sequences and masks are part of your data
sequences_train, masks_train = train_data
sequences_test, masks_test = test_data

def get_sequence_model(num_layers=2, num_units=16):
    class_vocab = label_processor.word_index  # Access vocabulary using word_index attribute

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

    # Bidirectional GRU with num_units units per direction
    x = Bidirectional(GRU(num_units, return_sequences=True))(frame_features_input, mask=mask_input)
    x = GRU(num_units)(x)  # Maintain the final GRU layer
    x = Dropout(0.4)(x)
    x = Dense(num_units, activation="relu")(x)
    output = Dense(len(class_vocab), activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

    rnn_model.compile(
        loss="sparse_categorical_crossentropy",
        optimizer="adam",
        metrics=["accuracy"]
    )
    return rnn_model

# Build the model
model = get_sequence_model()

# Compile the model
model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

# Train the model
model.fit(
    [sequences_train, masks_train],
    train_labels_numeric,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=([sequences_test, masks_test], test_labels_numeric),
)

# Evaluate the model on the test set
accuracy = model.evaluate([sequences_test, masks_test], test_labels_numeric)[1]
print(f"Test Accuracy: {accuracy}")


Epoch 1/30
5/5 [==============================] - 19s 1s/step - loss: 1.0987 - accuracy: 0.3208 - val_loss: 1.0986 - val_accuracy: 0.3333
Epoch 2/30
5/5 [==============================] - 0s 49ms/step - loss: 1.0986 - accuracy: 0.3333 - val_loss: 1.0986 - val_accuracy: 0.3333
Epoch 3/30
5/5 [==============================] - 0s 47ms/step - loss: 1.0986 - accuracy: 0.3333 - val_loss: 1.0986 - val_accuracy: 0.3333
Epoch 4/30
5/5 [==============================] - 0s 46ms/step - loss: 1.0986 - accuracy: 0.3333 - val_loss: 1.0986 - val_accuracy: 0.3333
Epoch 5/30
5/5 [==============================] - 0s 47ms/step - loss: 1.0986 - accuracy: 0.3333 - val_loss: 1.0986 - val_accuracy: 0.3333
Epoch 6/30
5/5 [==============================] - 0s 48ms/step - loss: 1.0987 - accuracy: 0.3333 - val_loss: 1.0986 - val_accuracy: 0.3333
Epoch 7/30
5/5 [==============================] - 0s 45ms/step - loss: 1.0986 - accuracy: 0.3333 - val_loss: 1.0986 - val_accuracy: 0.3333
Epoch 8/30
5/5 [============